In [1]:
import trimesh

scene = trimesh.Scene()

In [2]:
import torch
import numpy as np
import os
import shutil
from tqdm import tqdm
import yaml

import sys
import os

# models
from my_code.models.diag_conditional import DiagConditionedUnet
from diffusers import DDPMScheduler

import my_code.datasets.template_dataset as template_dataset

import my_code.diffusion_training_sign_corr.data_loading as data_loading

import networks.diffusion_network as diffusion_network
import matplotlib.pyplot as plt
import my_code.utils.plotting_utils as plotting_utils
import utils.fmap_util as fmap_util
import metrics.geodist_metric as geodist_metric
from my_code.sign_canonicalization.training import predict_sign_change
import argparse
from pyFM_fork.pyFM.refine.zoomout import zoomout_refine
import my_code.utils.zoomout_custom as zoomout_custom
from utils.shape_util import compute_geodesic_distmat
from my_code.diffusion_training_sign_corr.test.test_diffusion_cond import select_p2p_map_dirichlet, log_to_database, parse_args
import accelerate
import my_code.sign_canonicalization.test_sign_correction as test_sign_correction
import networks.fmap_network as fmap_network
from my_code.utils.median_p2p_map import dirichlet_energy


In [20]:
single_dataset, test_dataset = data_loading.get_val_dataset(
    'SHREC16_cuts_pair_noSingle', 'train', 200, preload=False, return_evecs=True, centering='bbox'
    )

In [10]:
def plot_p2p_map(scene, verts_x, faces_x, verts_y, faces_y, p2p, axes_color_gradient=[0, 1],
                 base_cmap='jet'):
    
    # assert axes_color_gradient is a list or tuple
    assert isinstance(axes_color_gradient, (list, tuple)), "axes_color_gradient must be a list or tuple"
    assert verts_y.shape[0] == len(p2p), f"verts_y {verts_y.shape} and p2p {p2p.shape} must have the same length"
    
    
    # normalize verts_x[:, 0] between 0 and 1
    # coords_x_norm = (verts_x[:, 0] - verts_x[:, 0].min()) / (verts_x[:, 0].max() - verts_x[:, 0].min())
    # coords_y_norm = (verts_x[:, 1] - verts_x[:, 1].min()) / (verts_x[:, 1].max() - verts_x[:, 1].min())
    # coords_z_norm = (verts_x[:, 2] - verts_x[:, 2].min()) / (verts_x[:, 2].max() - verts_x[:, 2].min())

    coords_x_norm = torch.zeros_like(verts_x)
    for i in range(3):
        coords_x_norm[:, i] = (verts_x[:, i] - verts_x[:, i].min()) / (verts_x[:, i].max() - verts_x[:, i].min())

    coords_interpolated = torch.zeros(verts_x.shape[0])
    for i in axes_color_gradient:
        coords_interpolated += coords_x_norm[:, i]
        
    # first colormap = interpolated y-axis values
    cmap = trimesh.visual.color.interpolate(coords_interpolated, base_cmap)
    
    # cmap = trimesh.visual.color.interpolate(verts_x[:, 0] + verts_x[:, 1], 'jet')
    
    # second colormap = first colormap values mapped to second mesh
    cmap2 = cmap[p2p].clip(0, 255)
    
    # diffuse material
    material=trimesh.visual.material.SimpleMaterial(
        image=None,
        diffuse=[245] * 4,
    )

    # add the first mesh
    mesh1 = trimesh.Trimesh(vertices=verts_x, faces=faces_x, validate=True)
    mesh1.visual.material = material
    mesh1.visual.vertex_colors = cmap[:len(mesh1.vertices)].clip(0, 255)
    scene.add_geometry(mesh1)
    
    
    
    # add the second mesh
    mesh2 = trimesh.Trimesh(vertices=verts_y + np.array([1, 0, 0]), faces=faces_y, validate=True)
    mesh2.visual.material = material
    mesh2.visual.vertex_colors = cmap2[:len(mesh2.vertices)]
    scene.add_geometry(mesh2)
    
    # scene.add_geometry(trimesh.creation.axis(origin_size=0.05))

    return scene

In [29]:
data = test_dataset[1]

scene.geometry.clear()

plot_p2p_map(
    scene,
    
    data['first']['verts'], data['first']['faces'],
    data['second']['verts'], data['second']['faces'],
    
    
    data['first']['corr'],
    # p2p_est_second[k].cpu(),
    axes_color_gradient=[0, 1],
    base_cmap='jet'
)

scene.show()